## BCM-826: Optimize CRM SFMC Datasets
##### Notebook to download example datasets for the last 7 days for BS/ON/JJ/VM in excel format
ticket: https://bestseller.jira.com/browse/BCM-820

### 0 Import packages & data

In [1]:
import os, sys
#figures directory
fig_dir = os.path.join(os.path.dirname(os.getcwd()), 'figures')
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

#packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency
import time
import matplotlib.ticker as plticker

#Analytics-toolkit
from database_connector.connector import DatabaseRedshift
from plotting.plotting import *
from bigquery_api.connector import BigQuery

#ga api
import googleanalytics as ga
import logging
from datetime import datetime
from dateutil.relativedelta import *
import calendar
import boto3


#settings 
%matplotlib inline
#plt.style.library['bsestyle_light']
plt.style.use('bsestyle_light')
pd.options.mode.chained_assignment = None

#make output tables wider and show more when columns has a lot of content (NO '...')
#pd.set_option('display.width', 1000)
#pd.set_option('max_colwidth', 200)

### 0. Data Preparation

In [2]:
#dictionary for accounts
bq_account_dict = {'BS': ['112804024',16,20], #account number, cd16, cd20
                   #'MM':'113556952',
                   #'JL':'113608165',
                   #'JR':'113608269',
                   'ON':['113635108',12,15],
                   'JJ':['113663276',12,15],
                   'VM':['113676549',11,14]}#,
                   #'VL':'113698901',
                   #'BI':'124406727',
                   #'NI':'113595925',
                   #'NM':'113606041',
                   #'OC':'113614154',
                   #'OS':'113613067',
                   #'PC':'113671938',
                   #'SL':'159210175',
                   #'YA':'113684854'}

In [3]:
bq = BigQuery()

### 1. Pages Dataset

In [4]:
# query
pages_query = """
                SELECT main.date,
                       main.site_brand,
                       main.session_code,
                       main.user_type,
                       main.channelgrouping,
                       main.page_url,
                       main.is_entry,
                       main.is_exit,
                       main.site_country,
                       main.page_type,
                       main.page_category,
                       sfmc.sfmc_id,
                       main.total_session_time,
                       ROUND(main.page_entry_time/1000,0) AS page_entry_time_seconds,
                       ROUND(main.page_exit_time/1000,0) AS page_exit_time_seconds,
                       ROUND(main.page_exit_time/1000,0) - ROUND(main.page_entry_time/1000,0) AS time_on_page 

                FROM (SELECT  concat(substr(date,1,4),"-",substr(date,5,2),"-",substr(date,7,2)) as date,
                              '{}' as site_brand,
                              CONCAT(CAST(visitId AS STRING), CAST(fullVisitorId AS STRING)) as session_code,
                              fullVisitorId, --need them to partition
                              visitId, --need them to partition
                              device.deviceCategory as user_type,

                              channelGrouping,
                              SPLIT(hits.page.pagepath, '?')[OFFSET(0)] as page_url,
                              IFNULL(hits.isEntrance,false) as is_entry,
                              IFNULL(hits.isexit,false) as is_exit,

                              max(if(cd.index = 1,cd.value,null)) as site_country,
                              MAX(CASE WHEN cd.index = 4 then cd.value end) as page_type,
                              MAX(CASE WHEN cd.index = {} then cd.value end) as page_category,

                              totals.timeonsite AS total_session_time, 
                              hits.time AS page_entry_time, 
                              IFNULL(LEAD(hits.time, 1) OVER (PARTITION BY fullVisitorId, visitId ORDER BY hits.time ASC),totals.timeonsite*1000) as page_exit_time

                        FROM `ga-360-bigquery-api.{}.ga_sessions_*`, 
                                  UNNEST(hits) AS hits,
                                  unnest(hits.customDimensions) as cd

                        WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                                and lower(device.browser) NOT LIKE '%app%'
                                and hits.type = 'PAGE'

                        GROUP BY 1,2,3,4,5,6,7,8,9,10,14,15) main

               
                LEFT JOIN (SELECT CONCAT(CAST(visitId AS STRING), CAST(fullVisitorId AS STRING)) as session_code,
                                    MAX(CASE WHEN cd.index = {} then cd.value END) as sfmc_id
                            FROM `ga-360-bigquery-api.{}.ga_sessions_*`, 
                                   UNNEST(hits) AS hits,
                                   unnest(hits.customDimensions) as cd
                            WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                                    and lower(device.browser) NOT LIKE '%app%'
                            GROUP BY 1
                            HAVING sfmc_id IS NOT NULL) sfmc ON main.session_code = sfmc.session_code

                WHERE sfmc.sfmc_id IS NOT NULL
                """

In [5]:
#import query
df_pages_appended = []
for account in bq_account_dict:
    dfi =bq.fetch(pages_query.format(account, bq_account_dict[account][1],bq_account_dict[account][0],bq_account_dict[account][2],bq_account_dict[account][0]))
    df_pages_appended.append(dfi)

df_pages = pd.concat(df_pages_appended)
df_pages = df_pages.reset_index(drop=True) 

In [6]:
#check if everything is okay
print(df_pages.info())
print(" ")
print(df_pages.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224916 entries, 0 to 224915
Data columns (total 16 columns):
date                       224916 non-null object
site_brand                 224916 non-null object
session_code               224916 non-null object
user_type                  224916 non-null object
channelgrouping            224916 non-null object
page_url                   224916 non-null object
is_entry                   224916 non-null bool
is_exit                    224916 non-null bool
site_country               224825 non-null object
page_type                  224916 non-null object
page_category              135391 non-null object
sfmc_id                    224916 non-null object
total_session_time         213041 non-null float64
page_entry_time_seconds    224916 non-null float64
page_exit_time_seconds     213094 non-null float64
time_on_page               213094 non-null float64
dtypes: bool(2), float64(4), object(10)
memory usage: 24.5+ MB
None
 
         date site_

In [7]:
#export
df_pages.to_excel("C:/ecommerce/ilaria/repos/projects/20191212-BCM826 - Optimize CRM datasets/data/pages_dataset.xlsx",index=False)

### 2. Product Dataset

In [8]:
product_query = """
                SELECT master.date,
                       master.site_brand,
                       master.session_code, 
                       master.user_type, 
                       master.site_country, 
                       master.channelGrouping, 
                       master.campaign, 
                       sfmc.sfmc_id,

                        case when (master.cat = 'wishlist' and master.act LIKE '%add%') then 'ATW' 
                             when (master.cat = 'Ecommerce' and master.act = 'Add to Cart') then 'ATB' 
                             when (master.page_type = 'pdp' and master.hit_type = 'PAGE') then 'PDPv'
                             END as hit_type,

                        master.sku, 

                        REGEXP_EXTRACT(master.url, r'([0-9]{})') AS style_no,
                        if(conversions.gis > 0,1,0) AS order_flag

                FROM (
                      SELECT concat(substr(date,1,4),"-",substr(date,5,2),"-",substr(date,7,2)) as date,
                             '{}' as site_brand,
                              CONCAT(CAST(visitId AS STRING), CAST(fullVisitorId AS STRING)) as session_code,
                              device.deviceCategory as user_type,      
                              trafficsource.campaign as campaign,
                              channelGrouping,
                              hits.type as hit_type,
                              hits.eventinfo.eventcategory as cat,
                              hits.eventinfo.eventaction act,

                              CONCAT(prod.productsku) as sku,
                              hits.page.pagePath AS url,
                              max(if(cd.index = 1,cd.value,null)) as site_country,
                              MAX(CASE WHEN cd.index = 4 then cd.value end) as page_type

                      FROM `ga-360-bigquery-api.{}.ga_sessions_*`, 
                                UNNEST(hits) AS hits,
                                unnest(hits.product) as prod,
                                unnest(hits.customDimensions) as cd

                      WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY)))  
                            and lower(device.browser) NOT LIKE '%app%'

                      group by 1,2,3,4,5,6,7,8,9,10,11
                      having ((cat = 'wishlist' and act LIKE '%add%') or 
                              (cat = 'Ecommerce' and act = 'Add to Cart') or 
                                (page_type = 'pdp' and hit_type = 'PAGE'))) master

                --get all sessions with sfmc_id for the period of our choice
                LEFT JOIN (SELECT CONCAT(CAST(visitId AS STRING), CAST(fullVisitorId AS STRING)) as session_code,
                                    MAX(CASE WHEN cd.index = {} then cd.value END) as sfmc_id
                            FROM `ga-360-bigquery-api.{}.ga_sessions_*`, 
                                   UNNEST(hits) AS hits,
                                   unnest(hits.customDimensions) as cd
                            WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                                    and lower(device.browser) NOT LIKE '%app%'
                            GROUP BY 1
                            HAVING sfmc_id IS NOT NULL) sfmc ON master.session_code = sfmc.session_code

                -- check per each session_code and product sku if it was bought or not
                LEFT JOIN (SELECT CONCAT(CAST(visitId AS STRING), CAST(fullVisitorId AS STRING)) as session_code,
                                  prod.productsku AS sku,
                                  COUNT(hits.transaction.transactionid) AS gis

                          FROM `ga-360-bigquery-api.{}.ga_sessions_*`, 
                                    UNNEST(hits) AS hits,
                                    unnest(hits.product) as prod

                          WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                                and lower(device.browser) NOT LIKE '%app%'

                          group by 1,2) conversions ON conversions.session_code = master.session_code
                                                        AND conversions.sku=master.sku 

                WHERE sfmc.sfmc_id IS NOT NULL
                GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
                                """

In [9]:
#import query
df_product_appended = []
for account in bq_account_dict:
    dfi =bq.fetch(product_query.format('{8}',account, bq_account_dict[account][0],bq_account_dict[account][2],bq_account_dict[account][0],bq_account_dict[account][0]))
    df_product_appended.append(dfi)

df_product = pd.concat(df_product_appended)
df_product = df_product.reset_index(drop=True) 

In [10]:
print(df_product.info())
print(" ")
print(df_product.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71159 entries, 0 to 71158
Data columns (total 12 columns):
date               71159 non-null object
site_brand         71159 non-null object
session_code       71159 non-null object
user_type          71159 non-null object
site_country       71159 non-null object
channelGrouping    71159 non-null object
campaign           71159 non-null object
sfmc_id            71159 non-null object
hit_type           71159 non-null object
sku                71159 non-null object
style_no           70871 non-null object
order_flag         71159 non-null int64
dtypes: int64(1), object(11)
memory usage: 6.5+ MB
None
 
         date site_brand                   session_code user_type  \
0  2019-12-12         BS  15761076213546489067798625893    mobile   
1  2019-12-14         BS  15763211292691892903490666601    mobile   
2  2019-12-14         BS  15763408856735233690707928596   desktop   
3  2019-12-14         BS  15763420714617265646350840903    tablet 

In [11]:
df_product.to_excel("C:/ecommerce/ilaria/repos/projects/20191212-BCM826 - Optimize CRM datasets/data/product_dataset.xlsx",index=False)

### 3. Internal Search Dataset

In [12]:
search_query = """
                SELECT main.*,
                       sub.internal_search_query
                FROM (SELECT concat(substr(date,1,4),"-",substr(date,5,2),"-",substr(date,7,2)) as date,
                            '{}' AS site_brand,
                             CONCAT(CAST(visitId AS STRING),CAST(fullVisitorId AS STRING)) AS session_code,
                             device.deviceCategory AS user_type,
                             MAX(IF(cd.index=1,cd.value,NULL)) AS site_country,
                             MAX(IF(cd.index={},cd.value,NULL)) AS sfmc_id,
                             channelGrouping,
                             trafficsource.campaign as campaign
                      FROM `ga-360-bigquery-api.{}.ga_sessions_*`, --BS.com
                             UNNEST(hits) as hits,
                             UNNEST(hits.customDimensions) AS cd
                      WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                             AND lower(device.browser) NOT LIKE '%app%' 
                             AND totals.pageviews > 0 
                      GROUP BY 1,2,3,4,7,8) main

                LEFT JOIN (SELECT CONCAT(CAST(visitId AS STRING),CAST(fullVisitorId AS STRING)) AS session_code,
                                  hits.page.searchkeyword  AS internal_search_query 
                           FROM `ga-360-bigquery-api.{}.ga_sessions_*`, --BS.com
                                  UNNEST(hits) as hits
                           WHERE (_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY))) 
                                  AND lower(device.browser) NOT LIKE '%app%' --excluding app
                                  AND totals.pageviews > 0 
                                  AND hits.type = 'PAGE'
                           GROUP BY 1,2
                           HAVING internal_search_query IS NOT NULL) sub ON main.session_code = sub.session_code

                WHERE sfmc_id IS NOT NULL
                      AND internal_search_query IS NOT NULL
                """

In [13]:
#import query
df_search_appended = []
for account in bq_account_dict:
    dfi =bq.fetch(search_query.format(account, bq_account_dict[account][2], bq_account_dict[account][0], bq_account_dict[account][0]))
    df_search_appended.append(dfi)

df_search = pd.concat(df_search_appended)
df_search = df_search.reset_index(drop=True) 

In [14]:
print(df_search.info())
print(" ")
print(df_search.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 9 columns):
date                     2211 non-null object
site_brand               2211 non-null object
session_code             2211 non-null object
user_type                2211 non-null object
site_country             2211 non-null object
sfmc_id                  2211 non-null object
channelGrouping          2211 non-null object
campaign                 2211 non-null object
internal_search_query    2211 non-null object
dtypes: object(9)
memory usage: 155.6+ KB
None
 
         date site_brand                   session_code user_type  \
0  2019-12-14         BS  15763192741207173982582553401    mobile   
1  2019-12-14         BS  15763192741207173982582553401    mobile   
2  2019-12-14         BS  15763394493808427788763203577   desktop   
3  2019-12-14         BS   1576339224411391502612748695    mobile   
4  2019-12-14         BS  15763298193010272047381258225    mobile   

  site_country  

In [15]:
df_search.to_excel("C:/ecommerce/ilaria/repos/projects/20191212-BCM826 - Optimize CRM datasets/data/internal_search_dataset.xlsx",index=False)